In [1]:
# Cell 1: Imports and Setup (People's Speech Focus + Cache Redirect)
import subprocess
import pandas as pd
from pathlib import Path
import glob
import os
from tqdm import tqdm
from datasets import load_dataset, Audio # Import datasets library
import soundfile as sf # Use soundfile for saving WAV
import shutil # Import shutil for moving files
import numpy as np # For audio array check
import random # For shuffling

# --- Configuration ---

# Base directory (assuming this notebook is in the 'ml' folder)
root_dir = Path('../')
data_dir = root_dir / 'data'

# --- !!! HF Cache Directory !!! ---
# Ensure this is set correctly to where your Parquet files are!
new_cache_dir = root_dir / "hf_cache"
os.environ['HF_HOME'] = str(new_cache_dir.resolve())
print(f"INFO: Using Hugging Face cache directory (HF_HOME): {os.environ['HF_HOME']}")
# Path where the actual downloaded parquet files likely are
# Adjust 'clean' and 'train' if you used a different config/split
ps_cache_data_path = new_cache_dir / "hub" / "datasets--MLCommons--peoples_speech" / "snapshots" / "f10597c5d3d3a63f8b6827701297c3afdf178272" / "clean"
print(f"INFO: Expecting Parquet files for People's Speech in: {ps_cache_data_path}")


# Processed output location for People's Speech controls
processed_ps_dir = data_dir / 'processed' / 'controls' / 'peoples_speech'
processed_audio_dir = processed_ps_dir / 'audio'
processed_metadata_dir = processed_ps_dir / 'metadata'

# Target audio format
TARGET_SAMPLE_RATE = 16000
AUDIO_FORMAT = "wav"
BYTES_PER_SAMPLE = 2 # For 16-bit WAV

# Target total size for the control dataset (in GB)
TARGET_SIZE_GB = 30
TARGET_SIZE_BYTES = TARGET_SIZE_GB * (1024**3)

# People's Speech config and split used during download attempt
PS_CONFIG = "clean"
PS_SPLIT = "train"

# --- Ensure directories exist ---
processed_audio_dir.mkdir(parents=True, exist_ok=True)
processed_metadata_dir.mkdir(parents=True, exist_ok=True)

print(f"Processed People's Speech Output Dir: {processed_ps_dir}")
print(f"Target Control Data Size: {TARGET_SIZE_GB} GB ({TARGET_SIZE_BYTES} bytes)")
print(f"Using People's Speech Config: '{PS_CONFIG}', Split: '{PS_SPLIT}'")

INFO: Using Hugging Face cache directory (HF_HOME): D:\Cornell\Academic\Spring 2025\Startup Studio\MVP\neurotone\ml\hf_cache
INFO: Expecting Parquet files for People's Speech in: ..\hf_cache\hub\datasets--MLCommons--peoples_speech\snapshots\f10597c5d3d3a63f8b6827701297c3afdf178272\clean
Processed People's Speech Output Dir: ..\data\processed\controls\peoples_speech
Target Control Data Size: 30 GB (32212254720 bytes)
Using People's Speech Config: 'clean', Split: 'train'


In [2]:
# Cell 2: Find Available Parquet Files and Estimate Sample Sizes

print("\n--- Finding available Parquet files and estimating potential WAV sizes ---")

# Find the parquet files that were actually downloaded in the cache
available_parquet_files = glob.glob(str(ps_cache_data_path / "*.parquet"))

if not available_parquet_files:
    raise FileNotFoundError(f"No Parquet files found in the expected cache location: {ps_cache_data_path}. Cannot proceed.")

print(f"Found {len(available_parquet_files)} Parquet files to process.")

all_samples_info = []
total_estimated_size = 0

# Iterate through the found parquet files to get metadata
for pq_file in tqdm(available_parquet_files, desc="Reading Parquet Metadata"):
    try:
        df_pq = pd.read_parquet(pq_file, columns=['id', 'duration_ms']) # Only load needed columns
        for index, row in df_pq.iterrows():
            duration_s = row['duration_ms'] / 1000.0
            # Estimate size of 16kHz, 16-bit mono WAV
            estimated_wav_size = duration_s * TARGET_SAMPLE_RATE * BYTES_PER_SAMPLE
            all_samples_info.append({
                'id': row['id'],
                'duration_ms': row['duration_ms'],
                'estimated_wav_size_bytes': estimated_wav_size,
                'source_parquet': Path(pq_file).name # Track source for potential debugging
            })
            total_estimated_size += estimated_wav_size
    except Exception as e:
        print(f"Warning: Failed to read or process {pq_file}: {e}")

total_estimated_gb = total_estimated_size / (1024**3)
print(f"\nFound metadata for {len(all_samples_info)} total samples across available Parquet files.")
print(f"Total estimated size if all were converted to {TARGET_SAMPLE_RATE}Hz mono WAV: {total_estimated_gb:.2f} GB")

if not all_samples_info:
    raise ValueError("No valid sample metadata could be extracted from the Parquet files.")


--- Finding available Parquet files and estimating potential WAV sizes ---
Found 231 Parquet files to process.


Reading Parquet Metadata: 100%|█| 231/231 [00:24<00:00, 


Found metadata for 431480 total samples across available Parquet files.
Total estimated size if all were converted to 16000Hz mono WAV: 177.23 GB


In [4]:
# Cell 3: Shuffle and Select Subset Based on Target Size

print(f"\n--- Selecting a random subset targeting ~{TARGET_SIZE_GB} GB ---")

# Make sure all_samples_info exists from Cell 2
if 'all_samples_info' not in locals() or not all_samples_info:
     raise NameError("Variable 'all_samples_info' not found or is empty. Ensure Cell 2 ran successfully.")

# Shuffle the list of all potential samples
random.shuffle(all_samples_info)

selected_samples_for_processing = []
current_selected_size = 0

for sample_info in tqdm(all_samples_info, desc="Selecting Samples"):
    # Add samples until we reach or exceed the target size
    if current_selected_size < TARGET_SIZE_BYTES:
        selected_samples_for_processing.append(sample_info)
        # Ensure the estimated size is treated as a number
        current_selected_size += float(sample_info.get('estimated_wav_size_bytes', 0))
    # Keep adding samples *until* the target size is met or exceeded
    # This ensures we get at least TARGET_SIZE_BYTES unless all samples are exhausted
    if current_selected_size >= TARGET_SIZE_BYTES:
         break


selected_ids_set = {s['id'] for s in selected_samples_for_processing}
final_selected_size_gb = current_selected_size / (1024**3)

print(f"\nSelected {len(selected_samples_for_processing)} samples with an estimated total size of {final_selected_size_gb:.2f} GB.")
if not selected_samples_for_processing:
      raise ValueError("Selection resulted in zero samples. Check target size and estimated sizes.")


--- Selecting a random subset targeting ~30 GB ---


Selecting Samples:  17%|▏| 72958/431480 [00:00<00:00, 10


Selected 72959 samples with an estimated total size of 30.00 GB.


In [6]:
# Cell 4: Process ONLY the Selected People's Speech Samples (Iterative Loading)

print("\n--- Loading and Preprocessing the SELECTED People's Speech Audio (Iterative Approach) ---")

# Ensure selected_ids_set exists from Cell 3
if 'selected_ids_set' not in locals() or not selected_ids_set:
    raise NameError("Variable 'selected_ids_set' not found or is empty. Ensure Cell 3 ran successfully.")
if 'available_parquet_files' not in locals() or not available_parquet_files:
     raise NameError("Variable 'available_parquet_files' not found or is empty. Ensure Cell 2 ran successfully.")

processed_metadata_list = []
preprocess_errors = []
skipped_count = 0
processed_count = 0
processed_ids_tracker = set() # Keep track of IDs we have processed/skipped

# Define the processing function (same as before)
def process_audio_sample_ps(sample):
    global processed_count, skipped_count, preprocess_errors, processed_ids_tracker

    try:
        sample_id = sample['id']
        # Construct the expected unique output filename
        output_filename = f"ps_{sample_id.replace('/', '_').replace('.', '_')}.{AUDIO_FORMAT}"
        output_path = processed_audio_dir / output_filename

        # Double check if we somehow already processed this ID in this run
        if sample_id in processed_ids_tracker:
             skipped_count +=1
             return None # Already handled

        # Check if the final WAV file already exists
        if output_path.exists():
            skipped_count += 1
            processed_ids_tracker.add(sample_id)
            # Create metadata entry for existing file
            return {
                "processed": True,
                "sample_id": sample_id,
                "speaker_id": sample_id,
                "label": 0,
                "relative_audio_path": str(output_path.relative_to(processed_ps_dir)),
                "original_corpus": "PeopleSpeech",
                "original_split": PS_SPLIT,
                "duration_ms": sample.get('duration_ms') # Duration might be missing if only checking file
            }

        # --- If WAV doesn't exist, proceed to load audio and process ---
        # Load audio array - THIS IS THE POINT WHERE datasets loads from cache/source
        # Ensure the 'audio' column exists and is loadable
        if 'audio' not in sample or sample['audio'] is None:
             raise ValueError("Audio data missing or null in sample.")

        audio_data = sample['audio']
        waveform = audio_data['array']
        original_sr = audio_data['sampling_rate']

        # Ensure waveform is numpy array
        if not isinstance(waveform, np.ndarray):
            waveform = np.array(waveform)

        # Ensure mono
        if waveform.ndim > 1:
            if waveform.shape[0] == 1: waveform = waveform[0]
            elif waveform.shape[1] == 1: waveform = waveform[:,0]
            else: waveform = waveform.mean(axis=1 if waveform.shape[1] < waveform.shape[0] else 0)

        # Resample if necessary using ffmpeg via subprocess
        resampled_waveform = waveform
        if original_sr != TARGET_SAMPLE_RATE:
            # Unique temp file names
            temp_id = output_filename.replace('.','_').replace('/','_')
            temp_input_path = processed_audio_dir / f"temp_{temp_id}_in.wav"
            temp_output_path = processed_audio_dir / f"temp_{temp_id}_out.wav"
            temp_input_path.parent.mkdir(parents=True, exist_ok=True)
            try:
                sf.write(temp_input_path, waveform, original_sr)
                command = [
                    "ffmpeg", "-i", str(temp_input_path),
                    "-ar", str(TARGET_SAMPLE_RATE), "-ac", "1",
                    "-vn", "-loglevel", "error", "-y", str(temp_output_path)
                ]
                subprocess.run(command, check=True, capture_output=True, timeout=180)
                resampled_waveform, sr_check = sf.read(temp_output_path, dtype='float32')
                if sr_check != TARGET_SAMPLE_RATE:
                     raise ValueError(f"ffmpeg post-read SR mismatch: {sr_check}")
            finally:
                 temp_input_path.unlink(missing_ok=True)
                 temp_output_path.unlink(missing_ok=True)

        # Save the processed audio
        sf.write(output_path, resampled_waveform, TARGET_SAMPLE_RATE)
        processed_count += 1
        processed_ids_tracker.add(sample_id) # Mark as processed in this run

        # Return metadata for the successfully processed file
        return {
            "processed": True,
            "sample_id": sample_id,
            "speaker_id": sample_id,
            "label": 0,
            "relative_audio_path": str(output_path.relative_to(processed_ps_dir)),
            "original_corpus": "PeopleSpeech",
            "original_split": PS_SPLIT,
            "duration_ms": sample.get('duration_ms')
         }

    except Exception as e:
        error_msg = f"Error processing sample with id {sample.get('id','N/A')}: {type(e).__name__} - {e}"
        preprocess_errors.append(error_msg)
        if 'output_path' in locals() and isinstance(output_path, Path) and output_path.exists():
            output_path.unlink(missing_ok=True)
        return {"processed": False, "error": error_msg}

# --- Iterate through available Parquet files, loading and processing selected samples ---
print(f"\nIterating through {len(available_parquet_files)} available Parquet files...")

samples_to_process_count = len(selected_ids_set)
print(f"Targeting {samples_to_process_count} selected sample IDs.")

# Wrap the loop with tqdm
with tqdm(total=samples_to_process_count, desc="Processing Selected Samples") as pbar:
    for pq_file_path in available_parquet_files:
        if len(processed_ids_tracker) >= samples_to_process_count:
             print("Already processed/found enough samples matching the selection. Stopping Parquet scan.")
             break # Optimization: stop if we already have enough

        try:
            # Load one parquet file
            ds_chunk = load_dataset("parquet", data_files={'train': str(pq_file_path)})['train']

            # Filter this chunk for IDs we care about
            chunk_ids = set(ds_chunk['id'])
            relevant_ids_in_chunk = selected_ids_set.intersection(chunk_ids)

            if relevant_ids_in_chunk:
                # Filter the dataset chunk
                ds_chunk_filtered = ds_chunk.filter(lambda x: x['id'] in relevant_ids_in_chunk, num_proc=1) # Filter first

                # Cast audio column ONLY for the filtered chunk before processing
                try:
                    ds_chunk_filtered = ds_chunk_filtered.cast_column("audio", Audio(decode=True)) # Decode audio
                except Exception as cast_e:
                     print(f"Warning: Failed to cast audio column for chunk from {Path(pq_file_path).name}: {cast_e}. Skipping chunk.")
                     continue # Skip this parquet file if casting fails

                # Process samples in this filtered chunk
                for sample in ds_chunk_filtered:
                    if sample['id'] in selected_ids_set and sample['id'] not in processed_ids_tracker:
                         result = process_audio_sample_ps(sample)
                         if result and result.get("processed"):
                              processed_metadata_list.append(result)
                              pbar.update(1) # Update progress bar per processed/skipped sample

        except Exception as e:
            print(f"ERROR processing Parquet file {Path(pq_file_path).name}: {e}")
            preprocess_errors.append(f"Parquet load/filter error: {Path(pq_file_path).name} - {e}")

        # Explicitly delete dataset chunk object to free memory
        if 'ds_chunk' in locals(): del ds_chunk
        if 'ds_chunk_filtered' in locals(): del ds_chunk_filtered
        import gc; gc.collect() # Force garbage collection


# Final Summary and Save Metadata
if processed_metadata_list:
    df_processed_controls = pd.DataFrame(processed_metadata_list)
    output_csv_path = processed_metadata_dir / "peoples_speech_controls_metadata_selected_30gb.csv"
    df_processed_controls.to_csv(output_csv_path, index=False)

    print(f"\n--- Audio Processing Summary ---")
    print(f"Selected {len(selected_ids_set)} samples based on ~{TARGET_SIZE_GB}GB target.")
    print(f"Found/Processed {len(processed_metadata_list)} corresponding audio files.")
    print(f"Processed {processed_count} new files.")
    print(f"Skipped {skipped_count} files that already existed (or processed in this run).")
    print(f"Encountered {len(preprocess_errors)} errors during processing attempts.")
    if preprocess_errors:
        print("Sample Errors:")
        for err in preprocess_errors[:20]: # Print more errors if needed
            print(f"- {err}")
    print(f"Processed control metadata (subset) saved to: {output_csv_path}")
    print("\n--- Sample Processed Control Metadata ---")
    print(df_processed_controls.head())
else:
    print("\nNo audio samples were successfully processed or found from the selected subset.")

print("\n--- People's Speech control data preparation finished ---")


--- Loading and Preprocessing the SELECTED People's Speech Audio (Iterative Approach) ---

Iterating through 231 available Parquet files...
Targeting 72959 selected sample IDs.


Processing Selected Samples:   0%| | 0/72959 [00:00<?, ?

Generating train split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/1868 [00:00<?, ? examples/s]

Processing Selected Samples:   0%| | 315/72959 [00:15<09

...

ERROR processing Parquet file train-00229-of-00804.parquet: An error occurred while generating the dataset


Generating train split: 0 examples [00:00, ? examples/s]

Processing Selected Samples:  95%|▉| 69152/72959 [45:34<

ERROR processing Parquet file train-00230-of-00804.parquet: An error occurred while generating the dataset



--- Audio Processing Summary ---
Selected 72959 samples based on ~30GB target.
Found/Processed 69152 corresponding audio files.
Processed 69151 new files.
Skipped 1 files that already existed (or processed in this run).
Encountered 17 errors during processing attempts.
Sample Errors:
- Error processing sample with id 42313BoardOfSelectmen_SLASH_4:23:13_SPACE_Board_SPACE_of_SPACE_Selectmen_DOT_mp3_00013.flac: LibsndfileError - Error opening '..\\data\\processed\\controls\\peoples_speech\\audio\\ps_42313BoardOfSelectmen_SLASH_4:23:13_SPACE_Board_SPACE_of_SPACE_Selectmen_DOT_mp3_00013_flac.wav': System error.
- Error processing sample with id 42313BoardOfSelectmen_SLASH_4:23:13_SPACE_Board_SPACE_of_SPACE_Selectmen_DOT_mp3_00014.flac: LibsndfileError - Error opening '..\\data\\processed\\controls\\peoples_speech\\audio\\ps_42313BoardOfSelectmen_SLASH_4:23:13_SPACE_Board_SPACE_of_SPACE_Selectmen_DOT_mp3_00014_flac.wav': System error.
- Error processing sample with id 42313BoardOfSelectmen_

In [7]:
# Cell 5: Combine Metadata and Create Final Splits

import pandas as pd
from pathlib import Path
from sklearn.model_selection import GroupShuffleSplit
import numpy as np

print("--- Combining and Splitting Final Dataset ---")

# --- Define Paths ---
root_dir = Path('../')
data_dir = root_dir / 'data'
processed_dir = data_dir / 'processed' # Main processed dir for outputs

# Input Metadata Paths
ps_controls_meta_path = processed_dir / 'controls' / 'peoples_speech' / 'metadata' / 'peoples_speech_controls_metadata_selected_30gb.csv'
# !! Adjust path if you saved Dementia Diaries metadata elsewhere !!
diaries_dem_meta_path = data_dir / 'raw' / 'dementia_diaries' / 'dementia_diaries_metadata.csv'
# Original DementiaNet metadata (assuming they are in processed_dir from the first notebook)
dnet_train_meta_path = processed_dir / 'train_metadata.csv'
dnet_val_meta_path = processed_dir / 'val_metadata.csv'
dnet_test_meta_path = processed_dir / 'test_metadata.csv'

# Output Paths for final splits
final_train_meta_path = processed_dir / 'train_meta_balanced.csv'
final_val_meta_path = processed_dir / 'val_meta_balanced.csv'
final_test_meta_path = processed_dir / 'test_meta_balanced.csv'

# --- Load Individual Metadata Files ---
try:
    df_ps_controls = pd.read_csv(ps_controls_meta_path)
    print(f"Loaded {len(df_ps_controls)} People's Speech control samples.")

    df_diaries_dem = pd.read_csv(diaries_dem_meta_path)
    print(f"Loaded {len(df_diaries_dem)} Dementia Diaries dementia samples.")

    df_dnet_train = pd.read_csv(dnet_train_meta_path)
    df_dnet_val = pd.read_csv(dnet_val_meta_path)
    df_dnet_test = pd.read_csv(dnet_test_meta_path)
    df_dnet_all = pd.concat([df_dnet_train, df_dnet_val, df_dnet_test], ignore_index=True)
    df_dnet_controls = df_dnet_all[df_dnet_all['label'] == 0].copy()
    df_dnet_dementia = df_dnet_all[df_dnet_all['label'] == 1].copy()
    print(f"Loaded {len(df_dnet_controls)} DementiaNet control samples.")
    print(f"Loaded {len(df_dnet_dementia)} DementiaNet dementia samples.")

except FileNotFoundError as e:
    print(f"ERROR: Could not find one of the metadata files: {e}")
    raise # Stop execution if files are missing

# --- Prepare and Standardize Columns ---
# Select relevant columns and standardize names
# People's Speech: Need 'relative_audio_path', 'speaker_id', 'label'
# Dementia Diaries: Need 'relative_audio_path', 'diarist_name' (as speaker_id), 'label'
# DementiaNet: Need 'file_path' (needs adjusting), 'speaker_id', 'label'

df_ps_controls = df_ps_controls[['relative_audio_path', 'speaker_id', 'label']].copy()
df_ps_controls['relative_audio_path'] = 'controls/peoples_speech/' + df_ps_controls['relative_audio_path'] # Make path relative to processed_dir
df_ps_controls['original_corpus'] = 'PeopleSpeech'

df_diaries_dem = df_diaries_dem[['relative_audio_path', 'diarist_name', 'label']].copy()
df_diaries_dem.rename(columns={'diarist_name': 'speaker_id'}, inplace=True)
# !! Adjust diaries relative path based on where WAVs are saved !!
# Assuming they end up in data/processed/dementia_diaries/audio/ relative to notebook root
# Path should be relative to processed_dir for consistency
# Example: if wavs are in ../data/processed/dementia_diaries/audio
df_diaries_dem['relative_audio_path'] = 'dementia_diaries/' + df_diaries_dem['relative_audio_path']
df_diaries_dem['original_corpus'] = 'Diaries'

# DementiaNet paths need fixing - they point to original sources or processed splits
# We need paths relative to the processed_dir where the FINAL WAVs reside
# Assuming DNet files are in processed/train|validation|test/dementia|nodementia/
def adjust_dnet_path(row):
    label_dir = 'dementia' if row['label'] == 1 else 'nodementia'
    # Determine split from original path (this is fragile)
    if 'train' in row['file_path']: split = 'train'
    elif 'validation' in row['file_path']: split = 'validation'
    elif 'test' in row['file_path']: split = 'test'
    else: split = 'unknown_split' # Fallback
    # Use the file_name column if it exists and is just the basename
    filename = row.get('file_name', Path(row['file_path']).name)
    return f"{split}/{label_dir}/{filename}"

df_dnet_controls['relative_audio_path'] = df_dnet_all[df_dnet_all['label'] == 0].apply(adjust_dnet_path, axis=1)
df_dnet_dementia['relative_audio_path'] = df_dnet_all[df_dnet_all['label'] == 1].apply(adjust_dnet_path, axis=1)
df_dnet_controls = df_dnet_controls[['relative_audio_path', 'speaker_id', 'label']]
df_dnet_dementia = df_dnet_dementia[['relative_audio_path', 'speaker_id', 'label']]
df_dnet_controls['original_corpus'] = 'DNetControl'
df_dnet_dementia['original_corpus'] = 'DNetDementia'

# --- Combine All Metadata ---
df_all_controls = pd.concat([df_ps_controls, df_dnet_controls], ignore_index=True)
df_all_dementia = pd.concat([df_diaries_dem, df_dnet_dementia], ignore_index=True)

print(f"\nTotal Control Samples: {len(df_all_controls)}")
print(f"Total Dementia Samples: {len(df_all_dementia)}")

# --- Undersample Dementia Data to Match Controls ---
n_control = len(df_all_controls)
n_dementia = len(df_all_dementia)

if n_dementia > n_control:
    print(f"Undersampling Dementia data from {n_dementia} to {n_control} samples...")
    df_dementia_sampled = df_all_dementia.sample(n=n_control, random_state=42)
else:
    print("Warning: Fewer Dementia samples than Control samples. Using all Dementia samples.")
    # Optionally, undersample controls instead if desired:
    # df_all_controls = df_all_controls.sample(n=n_dementia, random_state=42)
    df_dementia_sampled = df_all_dementia

df_balanced = pd.concat([df_all_controls, df_dementia_sampled], ignore_index=True)
print(f"Created balanced dataset with {len(df_balanced)} total samples ({len(df_all_controls)} controls, {len(df_dementia_sampled)} dementia).")

# Ensure unique speaker IDs across corpora if necessary (e.g., prefixing)
# Example: df_balanced['speaker_id'] = df_balanced['original_corpus'] + '_' + df_balanced['speaker_id'].astype(str)
# For now, assume IDs are unique enough or collision risk is low

# --- Split Balanced Data (Train/Val/Test using Speaker Groups) ---
print("\nSplitting balanced data into Train/Validation/Test sets (80/10/10 split) by speaker...")

if df_balanced.empty:
     raise ValueError("Balanced dataframe is empty, cannot split.")
if 'speaker_id' not in df_balanced.columns:
     raise KeyError("Column 'speaker_id' not found for GroupShuffleSplit.")


# First split: 80% train, 20% temp (val+test)
gss_train_valtest = GroupShuffleSplit(n_splits=1, test_size=0.20, random_state=42)
train_idx, valtest_idx = next(gss_train_valtest.split(df_balanced, groups=df_balanced['speaker_id']))

df_train = df_balanced.iloc[train_idx]
df_valtest = df_balanced.iloc[valtest_idx]

# Second split: 50% of temp is validation, 50% is test (gives 10% overall)
gss_val_test = GroupShuffleSplit(n_splits=1, test_size=0.50, random_state=42)
val_idx, test_idx = next(gss_val_test.split(df_valtest, groups=df_valtest['speaker_id']))

df_val = df_valtest.iloc[val_idx]
df_test = df_valtest.iloc[test_idx]

# --- Save Final Metadata Splits ---
df_train.to_csv(final_train_meta_path, index=False)
df_val.to_csv(final_val_meta_path, index=False)
df_test.to_csv(final_test_meta_path, index=False)

print("\n--- Final Split Summary ---")
print(f"Train set: {len(df_train)} samples ({df_train['label'].sum()} dementia, {len(df_train)-df_train['label'].sum()} control) from {df_train['speaker_id'].nunique()} speakers.")
print(f"Validation set: {len(df_val)} samples ({df_val['label'].sum()} dementia, {len(df_val)-df_val['label'].sum()} control) from {df_val['speaker_id'].nunique()} speakers.")
print(f"Test set: {len(df_test)} samples ({df_test['label'].sum()} dementia, {len(df_test)-df_test['label'].sum()} control) from {df_test['speaker_id'].nunique()} speakers.")
print(f"Saved Train metadata to: {final_train_meta_path}")
print(f"Saved Validation metadata to: {final_val_meta_path}")
print(f"Saved Test metadata to: {final_test_meta_path}")

print("\n--- Data Combination and Splitting Finished ---")

--- Combining and Splitting Final Dataset ---
Loaded 69152 People's Speech control samples.
Loaded 1818 Dementia Diaries dementia samples.
Loaded 324 DementiaNet control samples.
Loaded 131 DementiaNet dementia samples.

Total Control Samples: 69476
Total Dementia Samples: 1949
Created balanced dataset with 71425 total samples (69476 controls, 1949 dementia).

Splitting balanced data into Train/Validation/Test sets (80/10/10 split) by speaker...

--- Final Split Summary ---
Train set: 57483 samples (1887 dementia, 55596 control) from 55612 speakers.
Validation set: 6976 samples (32 dementia, 6944 control) from 6951 speakers.
Test set: 6966 samples (30 dementia, 6936 control) from 6952 speakers.
Saved Train metadata to: ..\data\processed\train_meta_balanced.csv
Saved Validation metadata to: ..\data\processed\val_meta_balanced.csv
Saved Test metadata to: ..\data\processed\test_meta_balanced.csv

--- Data Combination and Splitting Finished ---


In [16]:
# from pathlib import Path
# import shutil

# src_dir = Path(r"D:\Cornell\Academic\Spring 2025\Startup Studio\MVP\neurotone\ml\data\processed\test\nodementia")
# dst_dir = Path(r"D:\Cornell\Academic\Spring 2025\Startup Studio\MVP\neurotone\ml\data\no_dementia")
# dst_dir.mkdir(parents=True, exist_ok=True)

# for file_path in src_dir.rglob('*'):
#     if not file_path.is_file():
#         continue
#     dest = dst_dir / file_path.name
#     if dest.exists():
#         # Skip duplicates by filename
#         continue
#     shutil.move(str(file_path), str(dest))